In [2]:
import numpy as np
import magpylib as magpy
from magpylib.magnet import Cuboid, Cylinder, CylinderSegment
from maggeometry import halbach_cylinder
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid

### Nelder-Mead Adaptive Optimization of Halbach 8 main
n = 8

### Samwise

In [3]:
# def halbach_cylinder(Br, c_d, c_h, D, n, alternate=False):

In [4]:
def get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target=10, use_z=False):
    mT_to_G = 10
    B = magnets.getB(grid)
    G = B * mT_to_G
    # find magnitude of the b-field at all points over grid
    Gmag = 0
    if use_z:
        Gmag = G[:,:,2]
    else:
        Gmag = np.linalg.norm(G, axis=2)
#     Gmag = np.sqrt(G[:,:,0]**2 + G[:,:,1]**2 + G[:,:,2]**2)
#     Gmag = G[:,:,2]
    # find magnitude of b-field at center of grid
    mid_id = int(grid_res/2)
    Gcenter = Gmag[mid_id][mid_id]
    # calculate nonuniformity with respect to center b-field
    Gnon = np.abs((Gmag - Gcenter)/Gcenter)
    Gtarget_non = np.abs((Gmag - target)/target)
#     Gtarget_non = np.abs((Gmag - target)/target)
#     max_nonuniformity = np.max(Gnon)
    av_nonuniformity = np.sum(Gnon) / grid_res**2
    av_target_nonuniformity = np.sum(Gtarget_non) / grid_res**2
#     return Gmag, Gnon, Gcenter, av_nonuniformity, av_target_nonuniformity, max_nonuniformity
    return Gcenter, av_nonuniformity, av_target_nonuniformity

In [5]:
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid

In [6]:
n = 8
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
Br = 1.09e3
c_h = 640
threshold = 0.1 # in G
target = 10

          #c_d,     D
bounds = ((1, 20), (21, 300))

def obj1_nonuniformity(x):
    magnets = halbach_cylinder(Br, x[0], c_h, x[1], n)
    center_field, av_nonuniformity, av_target_nonuniformity = get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target)
    if (abs(center_field - target) > threshold):
        return av_target_nonuniformity
    return av_nonuniformity

In [7]:
options = {'disp': True, 'fatol': 1e-18, 'maxiter': 5e3, 'adaptive': True}
method = "Nelder-Mead"

shots = 25
results_x = []
results_fun = []
guesses_x0 = []
b_len = len(bounds)
for i in range(shots):
    x0 = [0, 0]
    for b in range(b_len):
        x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
    print(x0)
    res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)
    guesses_x0.append(x0)
    results_x.append(res.x)
    results_fun.append(res.fun)

print("Objective function values:")
print(results_fun)
print("Result parameters:")
print(results_x)
print("Starting guesses:")
print(guesses_x0)

[4.745769614545185, 202.99212436803114]


/var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_20935/1123013417.py:14: RuntimeWarning: Maximum number of iterations has been exceeded.
  res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)


[19.49931149141057, 58.937701093661026]
[15.204848804960635, 299.4000676651341]
Optimization terminated successfully.
         Current function value: 0.000027
         Iterations: 95
         Function evaluations: 228
[18.618656798513243, 299.5139273324633]
[5.570238141635976, 143.46938412698637]
[3.168749799211564, 88.93950372456605]
Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 92
         Function evaluations: 239
[5.549019446458098, 227.733171417172]
Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 144
         Function evaluations: 331
[8.969083380037455, 24.343199928112412]
[9.514976712867218, 91.71698436815662]
[5.788834093882194, 284.2852936421551]
Optimization terminated successfully.
         Current function value: 0.000027
         Iterations: 83
         Function evaluations: 206
[14.979518168089704, 209.68603393630502]
Optimization terminated successfully.
         

In [8]:
print(results_x)

[array([  1.81408834, 120.10409334]), array([  1.82711715, 120.11376491]), array([  4.59593312, 300.        ]), array([  4.59548858, 300.        ]), array([  1.81431257, 120.13895327]), array([  1.82359528, 120.12267313]), array([  1.82366815, 120.14971748]), array([ 1.       , 66.3615305]), array([  1.82040839, 120.13744377]), array([  4.59516116, 300.        ]), array([  4.59582318, 300.        ]), array([  1.8224266 , 120.13003909]), array([  4.5959355, 300.       ]), array([  1.82057706, 120.14483816]), array([  4.5955174, 300.       ]), array([  1.81880048, 120.13821613]), array([ 1.        , 66.36152757]), array([  1.82296389, 120.11954623]), array([  1.82094653, 120.14472709]), array([  1.81877578, 120.13998533]), array([  4.57441615, 298.62413039]), array([  1.821527  , 120.16772069]), array([ 1.        , 66.36153017]), array([  4.59553698, 300.        ]), array([ 1.        , 66.36152984])]


In [9]:
charr = np.full((1, shots), 640).flatten()
r_n = np.full((1, shots), n).flatten()

In [10]:
from maggeometry import halbach_cylinder
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid, get_grid_mag_and_nonuniformity
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results_g_center = []
results_nonun = []
for x in results_x:
    magnets = halbach_cylinder(Br, x[0], charr[0], x[1], n)
    Gmag, Gnon, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_mag_and_nonuniformity(magnets, grid, grid_res)
    results_g_center.append(center_field)
    results_nonun.append(av_nonuniformity)

In [11]:
import pandas as pd
def make_opt_res_halbach_csv(name, fun, g_center, r_n, r_ch, x, guesses):
    r_f = np.array([fun])
    r_g = np.array(guesses)
    r_gc = np.array([g_center])
    r_n = np.array([r_n])
    r_ch = np.array([r_ch])
    r_x = np.array(x)
    a = np.concatenate((r_f.T, r_gc.T, r_n.T, r_ch.T, r_x, r_g), axis=1)
    b = a[a[:, 0].argsort()]

    columns = ['nonuniformity', 'center_field_gauss', 'n_rods', 'cyl_height']

    columns.append('cyl_diameter')
    columns.append('D')
    columns.append('g_cyl_diameter')
    columns.append('g_D')
    
    
    columns = np.array(columns)
    b_df = pd.DataFrame(b, columns = columns)
    
    c = np.concatenate((r_f.T, r_x), axis=1)
    new_array = [tuple(row) for row in c]
    u = np.unique(new_array, axis=0)
    n_unique = len(u)
    print('Number of unique results:', n_unique)
    print(b_df)
    b_df.to_csv(name + str(n_unique) + '.csv')
    return b, b_df

import datetime
today = datetime.datetime.today().strftime('%Y_%m_%d')

b, b_df = make_opt_res_halbach_csv(f'results/halbach/samwise/{n}_{today}_samwise_halbach_main_1090mT_', results_fun, results_g_center, r_n, charr, results_x, guesses_x0)

Number of unique results: 25
    nonuniformity  center_field_gauss  n_rods  cyl_height  cyl_diameter  \
0        0.000001           10.040183     8.0       640.0      1.823668   
1        0.000001            9.988571     8.0       640.0      1.818800   
2        0.000001            9.988005     8.0       640.0      1.818776   
3        0.000001           10.006368     8.0       640.0      1.820408   
4        0.000001            9.939216     8.0       640.0      1.814313   
5        0.000001           10.013617     8.0       640.0      1.821527   
6        0.000001           10.043906     8.0       640.0      1.823595   
7        0.000001           10.037475     8.0       640.0      1.822964   
8        0.000001           10.029805     8.0       640.0      1.822427   
9        0.000001           10.006989     8.0       640.0      1.820577   
10       0.000001           10.011070     8.0       640.0      1.820947   
11       0.000001           10.084235     8.0       640.0      1.827117

In [12]:
m = halbach_cylinder(Br, 1.101233, 640, 68.228286, n)
Gmag, Gnon, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_mag_and_nonuniformity(m, grid, grid_res)
print(av_nonuniformity)

1.5573750827657786e-07
